# Rules of persona
+ Each sentence must contain between 4 and 20 words or punctuation marks.
+ It contains either the word I or my.
+ At least one verb, and (iv) at least one noun, pronoun or adjective.

# Example Dialogue with persona
- Persona: [“I like sport”, “I work a lot”]
- Context: “I love running.”
- Response: “Me too! But only on weekends.”

{"dialog": [["没有 钱   万万 不行 ！ ~"], ["现实 就是 如此"]], "profile": [{"tag": ["漫画;旅遊;星座"], "loc": "广东 广州", "gender": "male"}, {"tag": [""], "loc": "", "gender": ""}], "uid": [0, 1]}

{'dialog': ["For what it's worth, I don't have a problem with it.", 'My apologies.  I did not have any problems with it, but I will be more careful in the future.'], 'profile': [{'tag': ["for what it's worth, i don't have a problem with it."], 'loc': '', 'gender': ''}, {'tag': [' i did not have any problems with it, but i will be more careful in the future.'], 'loc': '', 'gender': ''}], 'uid': [0, 1]}

# test_data Example
{
    
    "uid": [0, 1, 2], 
    
    "dialog": [["剧烈运动 是 吧"], ["各种 剧烈运动"], ["... 姐 最近 有点 寂寞 过头 了 ..."]], 
    
    "responder_profile": {"loc": "海南", "gender": "female", "tag": "美食;宅;80后"}, 
    
    "profile": [{"loc": "天津 滨海新区", "gender": "male", "tag": ""}, {"loc": "海南", "gender": "female", "tag": "美食;宅;80后"}, {"loc": "安徽 合肥","gender": "male", "tag": "游戏动漫;双子座;宅;音乐;90后;WOW台服众"}], 
    
    "golden_response": ["可不是 ， 我 又 不 像 你 ， 有 女神 。"]

}

# Output Example
{
    
    "uid": [0], 
    
    "dialog": ["[\"For what it's worth, I don't have a problem with it.\"]"], 
    
    "responder_profile": {"loc": "", "gender": "", "tag": "['i did not have any problems with it, but i will be more careful in the future.']"}, 
    
    "profile": [{"loc": "", "gender": "", "tag": "[]"}], 
    
    "golden_response": "['My apologies.  I did not have any problems with it, but I will be more careful in the future.']"
    
}

# Constant Value

In [20]:
NPARTITIONS = 1000
INPUT_PATH = "./outputs/persona1.csv"
SCHEDULER = "threads"

# Imports

In [8]:
import pandas as pd 
import json
import bz2
from tqdm import tqdm
import glob
import dask.dataframe as dd
from dask.diagnostics import ProgressBar
import spacy
import os
#import neuralcoref

In [9]:
tqdm.pandas()
ProgressBar().register()
nlp = spacy.load('en_core_web_sm')
#neuralcoref.add_to_pipe(nlp)

In [10]:
df_input = pd.read_csv(INPUT_PATH)
df_input.head(5)

,Unnamed: 0,body,parent_body,original_body,original_parent_body,ups,author,persona,parent_persona,dialog,json
0,0,"[""For what it's worth, I don't have a problem ...",['My apologies. I did not have any problems w...,"For what it's worth, I don't have a problem wi...",My apologies. I did not have any problems wit...,2,0gleth0rpe,[],"['i did not have any problems with it, but i w...","[[""For what it's worth, I don't have a problem...","{'dialog': [[""For what it's worth, I don't hav..."
1,1,"[""Yeah I wish I could get the test some attent...","[""I took that 6 page survey and ended up where...",Yeah I wish I could get the test some attentio...,I took that 6 page survey and ended up where i...,1,24sparky,['i wish i could get the test some attention b...,['i took that 6 page survey and ended up where...,"[[""Yeah I wish I could get the test some atten...","{'dialog': [[""Yeah I wish I could get the test..."
2,2,['Happy New Year :)'],"['Thanks for the links, im looking through the...",Happy New Year :),"Thanks for the links, im looking through them ...",1,24sparky,[],"['thanks for the links, im looking through the...","[['Happy New Year :)'], ['Thanks for the links...","{'dialog': [['Happy New Year :)'], ['Thanks fo..."
3,3,"['Hmm, I came out as ’centrist’ but right betw...","[""World's Smallest Political Quiz:\r\nhttp://w...","Hmm, I came out as ’centrist’ but right betwee...",World's Smallest Political Quiz:\r\nhttp://www...,1,24sparky,"['hmm, i came out as ’centrist’ but right betw...",['personally i think nolan chart is more usefu...,"[['Hmm, I came out as ’centrist’ but right bet...","{'dialog': [['Hmm, I came out as ’centrist’ bu..."
4,4,"[""I wonder if it's more than an attitude towar...","[""The problem is not guns, it's attitude towar...",I wonder if it's more than an attitude towards...,"The problem is not guns, it's attitude towards...",8,24sparky,"[""i wonder if it's more than an attitude towar...",[],"[[""I wonder if it's more than an attitude towa...","{'dialog': [[""I wonder if it's more than an at..."


In [16]:
def create_json(row):
    return {
        "uid":[0],
        "dialog":[row["body"]],
        "responder_profile":{
            "loc":"",
            "gender":"",
            "tag":row["parent_persona"]
        },
        "profile":[
            {
                "loc":"",
                "gender":"",
                "tag":row["persona"]
            },
        ],
        "golden_response":row["parent_body"]  
    }

In [17]:
df_input["json"] = df_input.progress_apply(create_json, axis=1)
df_input.head(5)

100%|██████████| 3878/3878 [00:00<00:00, 28927.16it/s]


,Unnamed: 0,body,parent_body,original_body,original_parent_body,ups,author,persona,parent_persona,dialog,json
0,0,"[""For what it's worth, I don't have a problem ...",['My apologies. I did not have any problems w...,"For what it's worth, I don't have a problem wi...",My apologies. I did not have any problems wit...,2,0gleth0rpe,[],"['i did not have any problems with it, but i w...","[[""For what it's worth, I don't have a problem...","{'uid': [0], 'dialog': ['[""For what it's worth..."
1,1,"[""Yeah I wish I could get the test some attent...","[""I took that 6 page survey and ended up where...",Yeah I wish I could get the test some attentio...,I took that 6 page survey and ended up where i...,1,24sparky,['i wish i could get the test some attention b...,['i took that 6 page survey and ended up where...,"[[""Yeah I wish I could get the test some atten...","{'uid': [0], 'dialog': ['[""Yeah I wish I could..."
2,2,['Happy New Year :)'],"['Thanks for the links, im looking through the...",Happy New Year :),"Thanks for the links, im looking through them ...",1,24sparky,[],"['thanks for the links, im looking through the...","[['Happy New Year :)'], ['Thanks for the links...","{'uid': [0], 'dialog': ['['Happy New Year :)']..."
3,3,"['Hmm, I came out as ’centrist’ but right betw...","[""World's Smallest Political Quiz:\r\nhttp://w...","Hmm, I came out as ’centrist’ but right betwee...",World's Smallest Political Quiz:\r\nhttp://www...,1,24sparky,"['hmm, i came out as ’centrist’ but right betw...",['personally i think nolan chart is more usefu...,"[['Hmm, I came out as ’centrist’ but right bet...","{'uid': [0], 'dialog': ['['Hmm, I came out as ..."
4,4,"[""I wonder if it's more than an attitude towar...","[""The problem is not guns, it's attitude towar...",I wonder if it's more than an attitude towards...,"The problem is not guns, it's attitude towards...",8,24sparky,"[""i wonder if it's more than an attitude towar...",[],"[[""I wonder if it's more than an attitude towa...","{'uid': [0], 'dialog': ['[""I wonder if it's mo..."


In [18]:
list_json = df_input["json"].tolist()
with open(f"./outputs/test_data.json", "wt", encoding="utf-8") as file:
    for dic in list_json:
        file.write(str(json.dumps(dic))+"\n")

In [21]:
import subprocess
subprocess.run(['jupyter', 'nbconvert', '--to', 'script', '*.ipynb'])

CompletedProcess(args=['jupyter', 'nbconvert', '--to', 'script', '*.ipynb'], returncode=0)